In [35]:
import asana        
import pandas as pd 
import os

from datetime import datetime

In [110]:
client = asana.Client.access_token('1/1204862582358995:58225b495568b45ac37d588db2fea9a6')

# to surpress warnings
client.headers = {'Asana-Enable': 'new_goal_memberships','Asana-Enable': "new_user_task_lists"}

In [ ]:
# creating a directory to store all the fetch results
if not os.path.exists('./fetchedRecords'):
    os.makedirs('./fetchedRecords')

# Fetching Workspaces

In [82]:
workspaces = list(client.workspaces.find_all())
df_workspaces = pd.DataFrame(workspaces)
df_workspaces.to_csv('./fetchedRecords/workspaces.csv', index= False)
workspaces

[{'gid': '1204862567026473', 'name': 'My workspace', 'resource_type': 'workspace'}]


# Fetching Projects in a Workspace

In [123]:
# Getting the workspace ID for fetching projects
df_existedWorkspace = pd.read_csv('./fetchedRecords/workspace.csv')
workspaceIDs = list(df_existedWorkspace['gid'])
workspaceIDs

[1204862567026473]

In [87]:
projects = list(client.projects.find_all(workspaceIDs[0]))
df_projects = pd.DataFrame(projects)
df_projects.to_csv('./fetchedRecords/projects.csv', index= False)
projects

[{'gid': '1204862567620455',
  'name': 'Cross-functional project plan',
  'resource_type': 'project'}]

# Fetching Sections in a Project

In [116]:
# Getting the project ID for fetching sections and tasks
df_existedProjects = pd.read_csv('./fetchedRecords/projects.csv')
projectIDs = list(df_existedProjects['gid'])
projectIDs

[1204862567620455]

In [122]:
sections = list(client.sections.find_by_project(projectIDs[0]))
df_sections = pd.DataFrame(sections)
df_sections.to_csv('./fetchedRecords/sections.csv', index= False)
sections

[{'gid': '1204862567620456', 'name': 'To do', 'resource_type': 'section'},
 {'gid': '1204862567620458', 'name': 'Doing', 'resource_type': 'section'},
 {'gid': '1204862567620459', 'name': 'Done', 'resource_type': 'section'}]

# Fetching Tasks in a Project

In [108]:
# fetching only modifies tasks (optimized approach) 

current_datetime = datetime.utcnow().isoformat()

try: # file exists ==> some etl has already performed 
    df_tasks = pd.read_csv('./fetchedRecords/tasks.csv') # Assumed that file is always present
    # print(f'Previous df\n{df_tasks}\n')    
except FileNotFoundError: # etl running for the first time 
    df_tasks = pd.DataFrame(columns=['gid', 'name', 'resource_type', 'resource_subtype', 'etl_date_created', 'etl_date_modified'])
    past_date = datetime(2000, 1, 1, 00, 00, 0000)
    last_modified = past_date.strftime('%Y-%m-%dT%H:%M:%S.%f%z')
else:
    last_modified = df_tasks['etl_date_modified'].max()

tasks = client.tasks.find_by_project(projectIDs[0], modified_since=last_modified)
df_modifiedTasks = pd.DataFrame(tasks)
    
if len(df_modifiedTasks) == 0: # if no records are modifed since last_modified date 
    print('No modified records!!!')    
else:
    df_tasks['gid'] = df_tasks['gid'].astype(str)
    df_tasks = df_tasks[~df_tasks['gid'].isin(df_modifiedTasks['gid'])]

    # Adding etl_date_created and etl_date_modified columns in fetched df
    df_modifiedTasks['etl_date_modified'] = current_datetime
    df_modifiedTasks = df_modifiedTasks.merge(df_tasks[['gid', 'etl_date_created']], on='gid', how='left')
    df_modifiedTasks['etl_date_created'].fillna(current_datetime, inplace=True)
    # print(f'processed fetched df\n{df_modifiedTasks}\n')

    # merging the two dfs
    df_updatedTasks = pd.concat([df_tasks, df_modifiedTasks]).drop_duplicates(subset='gid', keep='last')
    # print(f'final df\n{df_updatedTasks}\n')
           
    df_updatedTasks.to_csv(f'./fetchedRecords/tasks.csv', index=False)

# Fetching task details 

In [111]:
current_datetime = datetime.utcnow().isoformat()

df_tasks = pd.read_csv('./fetchedRecords/tasks.csv')
tasksDetails_list = []

try: # file exists ==> some etl has already performed 
    df_existedTaskDetails = pd.read_csv('./fetchedRecords/taskDetails.csv') # Assumed that file is always present
    # print(f'Previous df\n{df_existedTaskDetails}\n')    
except FileNotFoundError: # etl running for the first time 
    # getting gids of all tasks 
    gids_tasksToFetch = list(df_tasks['gid'])
    df_existedTaskDetails = pd.DataFrame(columns=['gid','etl_date_created', 'etl_date_modified'])
else:
    # getting the gids of tasks which are modifed after taskDetail etl previous run
    df_taskAndTaskDetails = pd.merge(df_tasks, df_existedTaskDetails, on='gid', how = 'left', suffixes=('_df1', '_df2'))

    gids_modifiedTasks = list(df_taskAndTaskDetails[df_taskAndTaskDetails['etl_date_modified_df1'] > df_taskAndTaskDetails['etl_date_modified_df2']]['gid'])
    gids_newTasks = list(df_taskAndTaskDetails[df_taskAndTaskDetails['etl_date_modified_df2'].isnull()]['gid'])

    gids_tasksToFetch = gids_modifiedTasks + gids_newTasks  

if len(gids_tasksToFetch) == 0:
    print('No modified records!!!')
else:
    df_tasks['gid'] = df_tasks['gid'].astype(str)
    df_existedTaskDetails['gid'] = df_existedTaskDetails['gid'].astype(str)
   
    # calling API for the modified task details only
    for task_gid in gids_tasksToFetch:
        task = client.tasks.get_task(str(task_gid))
        tasksDetails_list.append(task)
    df_newTaskDetails = pd.DataFrame(tasksDetails_list)

    # Adding etl_date_created and etl_date_modified columns in fetched df
    df_newTaskDetails['etl_date_modified'] = current_datetime
    df_newTaskDetails = df_newTaskDetails.merge(df_existedTaskDetails[['gid', 'etl_date_created']], on='gid', how='left')
    df_newTaskDetails['etl_date_created'].fillna(current_datetime, inplace=True)    
    # print(f'processed fetched df\n{df_modifiedTasks.head()}\n')

    # merging the two dfs
    df_updatedTaskDetails = pd.concat([df_existedTaskDetails, df_newTaskDetails]).drop_duplicates(subset='gid', keep='last')
    # print(f'final df\n{df_updatedTaskDetails.head()}\n')
     
    df_updatedTaskDetails.to_csv(f'./fetchedRecords/taskDetails.csv', index=False)

No modified records!!!
